<a href="https://colab.research.google.com/github/yeboda96/Image-Classification-Project/blob/master/car_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import scipy.io
import shutil
import os


mat = scipy.io.loadmat('drive/My Drive/cars_train_annos.mat')
# print(mat['annotations'])
training_class = mat['annotations']['class']
training_fname = mat['annotations']['fname']
training_x1 = mat['annotations']['bbox_x1']
training_y1 = mat['annotations']['bbox_y1']
training_x2 = mat['annotations']['bbox_x2']
training_y2 = mat['annotations']['bbox_y2']

mat = scipy.io.loadmat('drive/My Drive/cars_test_annos_withlabels.mat')
print(mat['annotations'])
testing_class = mat['annotations']['class']
testing_fname = mat['annotations']['fname']
# print(testing_fname)
# print(testing_class)

training_source = 'drive/My Drive/cars_train/' # specify source training image path
training_output = 'drive/My Drive/train_folder/' # specify target trainig image path (trainig images need to be orgnized to specific structure)
for idx, cls in enumerate(training_class[0]):
    cls = cls[0][0]
    fname = training_fname[0][idx][0]
    # print(cls)
    output_path = os.path.join(training_output, str(cls))
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    shutil.copy(os.path.join(training_source, fname), os.path.join(output_path, fname))

testing_source = 'drive/My Drive/cars_test/' # specify source testing image path
testing_output = 'drive/My Drive/test_folder/' # specify target testing image path (testing images need to be orgnized to specific structure)
for idx, cls in enumerate(testing_class[0]):
    cls = cls[0][0]
    fname = testing_fname[0][idx][0]
    output_path = os.path.join(testing_output, str(cls))
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    shutil.copy(os.path.join(testing_source, fname), os.path.join(output_path, fname))

[[(array([[30]], dtype=uint8), array([[52]], dtype=uint8), array([[246]], dtype=uint8), array([[147]], dtype=uint8), array([[181]], dtype=uint8), array(['00001.jpg'], dtype='<U9'))
  (array([[100]], dtype=uint8), array([[19]], dtype=uint8), array([[576]], dtype=uint16), array([[203]], dtype=uint8), array([[103]], dtype=uint8), array(['00002.jpg'], dtype='<U9'))
  (array([[51]], dtype=uint8), array([[105]], dtype=uint8), array([[968]], dtype=uint16), array([[659]], dtype=uint16), array([[145]], dtype=uint8), array(['00003.jpg'], dtype='<U9'))
  ...
  (array([[33]], dtype=uint8), array([[27]], dtype=uint8), array([[602]], dtype=uint16), array([[252]], dtype=uint8), array([[17]], dtype=uint8), array(['08039.jpg'], dtype='<U9'))
  (array([[33]], dtype=uint8), array([[142]], dtype=uint8), array([[521]], dtype=uint16), array([[376]], dtype=uint16), array([[38]], dtype=uint8), array(['08040.jpg'], dtype='<U9'))
  (array([[77]], dtype=uint8), array([[73]], dtype=uint8), array([[506]], dtype=ui

In [0]:
from keras.preprocessing import image
from keras.applications import vgg16
from keras.applications import vgg19
from keras.applications import resnet50
from keras.applications import inception_v3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Model,Sequential
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint
import argparse
from time import time

from skimage import exposure, color

from keras import backend as K
K.set_image_dim_ordering('tf')

In [0]:
def init_model(train_dir,val_dir,batch_size=16,model_name='vgg16',num_class=2,img_size=224):
    """
    initialize cnn model and training and validation data generator
    parms:
        args: parsed commandline arguments
    return:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
    """
    

    print('loading the model and the pre-trained weights...')

    # load base model
    if model_name == 'vgg16':
        base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape = (224,224,3)) # need specify input_shape
        # this preprocess_input is the default preprocess func for given network, you can change it or implement your own 
        # use inception_v3 preprocess for vgg16, it seems that it works better than vgg16.preprocess_input
        preprocess_input = inception_v3.preprocess_input 
    
    elif model_name == 'inception_v3':
        base_model = inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape = (224,224,3)) # need specify input_shape
        preprocess_input = inception_v3.preprocess_input
    elif model_name == 'resnet50':
        base_model = resnet50.ResNet50(include_top=False, weights='imagenet', input_shape = (224,224,3)) # need specify input_shape
        preprocess_input = resnet50.preprocess_input

    
    
    
    
    # initalize training image data generator
    # you can also specify data augmentation here
    train_datagen = image.ImageDataGenerator(
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        # samplewise_center=True,
        # samplewise_std_normalization=True,
        # rescale=1./255,
        preprocessing_function=preprocess_input, # preprocess_input,
        # rotation_range=30,
        # shear_range=0.1,
        # zoom_range=0.1,
        # vertical_flip=True,
        horizontal_flip=True
        )

    # initalize validation image data generator
    # you can also specify data augmentation here
    validation_datagen = image.ImageDataGenerator(
        # samplewise_center=True,
        # samplewise_std_normalization=True
        # rescale=1./255
        preprocessing_function=preprocess_input # preprocess_input
        )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        # color_mode='grayscale', # 'rgb'
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        # color_mode='grayscale',  # 'rgb'
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

    # fix base_model layers
    for layer in base_model.layers:
        layer.trainable = False

    # added some customized layers for your own data
    x = base_model.output
    if model_name == 'vgg16':
        x = Flatten(name='flatten')(x)
        x = Dense(512, activation='relu', name='fc1-pretrain')(x)
        x = Dense(256, activation='relu', name='fc2-pretrain')(x)
        x = Dropout(0.5, name='dropout')(x)
        
    elif model_name == 'inception_v3':
        x = GlobalAveragePooling2D(name='avg_pool')(x)
        x = Dense(256, activation='relu', name='fc1-pretrain')(x)
    elif model_name == 'resnet50':
        x = GlobalAveragePooling2D(name='avg_pool')(x)
        x = Dense(256, activation='relu', name='fc1-pretrain')(x)
        
        
    

    # added softmax layer
    predictions = Dense(num_class, activation='softmax', name='predictions')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(), metrics=["accuracy"])

    return model, train_generator, validation_generator

In [0]:
def train(model, train_generator, validation_generator, num_class= 2,model_name='vgg16', batch_size=16, epochs=30, suffix='laioffer'):
    """
    train the model
    parms:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
        args: parsed command line arguments
    return:
    """
    # define number of steps/iterators per epoch
    stepsPerEpoch = train_generator.samples / batch_size
    validationSteps= validation_generator.samples / batch_size

    # save the snapshot of the model to local drive
    pretrain_model_name = 'pretrained_{}_{}_{}_{}.h5'.format(model_name, num_class, epochs, suffix)
    # visualize the training process
    tensorboard = TensorBoard(log_dir="logs/{}_pretrain_{}".format(model_name, time()), histogram_freq=0, write_graph=True)
    checkpoint = ModelCheckpoint(pretrain_model_name, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    callbacks_list = [checkpoint, tensorboard]

    model.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps)
    return model

In [0]:
my_model_name='inception_v3'
my_batch_size=32
my_epochs=20
model, train_generator, validation_generator = init_model(train_dir='drive/My Drive/train_folder', model_name=my_model_name, val_dir='drive/My Drive/test_folder',batch_size=my_batch_size, num_class=196)
my_model=train(model, train_generator, validation_generator, model_name=my_model_name, num_class=196, batch_size=my_batch_size,epochs=my_epochs)

loading the model and the pre-trained weights...
Found 8144 images belonging to 196 classes.
Found 8041 images belonging to 196 classes.
Epoch 1/20
  5/254 [..............................] - ETA: 1:08:03 - loss: 5.4896 - acc: 0.0000e+00

In [0]:
my_model.save('drive/My Drive/coerce_model.h5')
print('Saved model to disk')

Saved model to disk


In [0]:
def fine_tune(model,train_generator, validation_generator, model_name,\
              batch_size, epochs,num_class=196,suffix='laioffer'):
    """
    fine tune the model
    parms:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
        args: parsed command line arguments
    return:
    """
    # for specific architectures, define number of trainable layers
    if model_name == 'vgg16':
        trainable_layers = 6
    elif model_name == 'inception_v3':
        trainable_layers = 35
    elif model_name == 'resnet50':
        trainable_layers = 12

    for layer in model.layers[:-1*trainable_layers]:
        layer.trainable = False

    for layer in model.layers[-1*trainable_layers:]:
        layer.trainable = True

    finetune_model_name = 'finetuned_{}_{}_{}_{}.h5'.format(model_name, num_class, \
                                                            epochs, suffix)
    tensorboard = TensorBoard(log_dir="logs/{}_finetune_{}".format(model_name, time()), histogram_freq=0, write_graph=True)
    checkpoint = ModelCheckpoint(finetune_model_name, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    callbacks_list = [checkpoint, tensorboard]

    model.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.01, momentum=0.9),metrics=["accuracy"])

    stepsPerEpoch = train_generator.samples / batch_size
    validationSteps= validation_generator.samples / batch_size
    model.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=epochs + 50,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps)
    return model

In [0]:
my_model=fine_tune(my_model, train_generator, validation_generator, model_name=my_model_name\
          ,batch_size=my_batch_size,epochs=my_epochs)

Epoch 1/70
82/81 [==============================] - 233s 3s/step - loss: 1.9167 - acc: 0.5294 - val_loss: 6.3440 - val_acc: 0.0761

Epoch 00001: val_acc improved from -inf to 0.07611, saving model to finetuned_inception_v3_196_20_laioffer.h5
Epoch 2/70
82/81 [==============================] - 221s 3s/step - loss: 1.8273 - acc: 0.5670 - val_loss: 6.2699 - val_acc: 0.0755

Epoch 00002: val_acc did not improve from 0.07611
Epoch 3/70
82/81 [==============================] - 217s 3s/step - loss: 1.7690 - acc: 0.5773 - val_loss: 6.2411 - val_acc: 0.0764

Epoch 00003: val_acc improved from 0.07611 to 0.07636, saving model to finetuned_inception_v3_196_20_laioffer.h5
Epoch 4/70
82/81 [==============================] - 217s 3s/step - loss: 1.7439 - acc: 0.5788 - val_loss: 6.2277 - val_acc: 0.0775

Epoch 00004: val_acc improved from 0.07636 to 0.07748, saving model to finetuned_inception_v3_196_20_laioffer.h5
Epoch 5/70
82/81 [==============================] - 211s 3s/step - loss: 1.7008 - acc:

In [0]:
model_json = my_model.to_json()
with open("drive/My Drive/tunned_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
my_model.save_weights("drive/My Drive/tunned_model.h5")
print("Saved model to disk")

In [0]:
from keras.preprocessing import image
from keras.applications import vgg16
from keras.applications import vgg19
from keras.applications import resnet50
from keras.applications import inception_v3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Model, Sequential, load_model
from keras import optimizers 
from keras.callbacks import TensorBoard, ModelCheckpoint

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from tqdm import tqdm
import numpy as np
import argparse
import os
import glob

from keras import backend as K
K.set_image_dim_ordering('tf')
# K.set_learning_phase(0)

Using TensorFlow backend.


In [0]:
def predict(model_name,test_dir,model_weight_name,img_size=224):
    # load preprocess func
    if model_name == 'vgg19':
        # base_model = vgg19.VGG19(include_top=False, weights=None, input_shape = (224,224,3)) # need specify input_shape
        preprocess_input = vgg19.preprocess_input
    elif model_name == 'inception_v3':
        # base_model = inception_v3.InceptionV3(include_top=False, weights=None, input_shape = (224,224,3)) # need specify input_shape
        preprocess_input = inception_v3.preprocess_input
    elif model_name == 'resnet50':
        # base_model = resnet50.ResNet50(include_top=False, weights=None, input_shape = (224,224,3)) # need specify input_shape
        preprocess_input = resnet50.preprocess_input

    test_datagen = image.ImageDataGenerator(
        preprocessing_function=preprocess_input
        )
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=1,
        shuffle=False,
        class_mode='categorical')

    # fnames = test_generator.filenames
    # label_map = test_generator.class_indices
    true_labels = test_generator.classes

    # load model
    model = load_model(model_weight_name)

    predicted_label_probs = model.predict_generator(test_generator, verbose=1)
    predicted_labels = np.argmax(predicted_label_probs, axis=1)

    # print(confusion_matrix(true_labels, predicted_labels))
    # print(classification_report(true_labels, predicted_labels))
    print("Accuracy = ", accuracy_score(true_labels, predicted_labels))

In [0]:
predict(model_name='inception_v3',test_dir='drive/My Drive/test_folder',model_weight_name='drive/My Drive/model.h5')

Found 8041 images belonging to 196 classes.


ValueError: ignored